In [ ]:
!nvidia-smi

In [ ]:
import joblib

In [ ]:
import sys
sys.path.append('utils/PyMO/')

In [ ]:
import sys
sys.path.append('utils/PyMO/pymo/')

In [ ]:
from pymo.parsers import BVHParser
import writers

In [ ]:
import numpy as np
import random

In [ ]:
import os

In [ ]:
p = BVHParser()

data_all = [(path, p.parse('data/lafan1/{}'.format(path))) for path in os.listdir('data/lafan1/') if 'subject' in path]

In [ ]:
train_actors = ['subject1', 'subject2', 'subject3', 'subject4']
test_actors = ['subject5']

In [ ]:
from scipy.spatial.transform import Rotation as R

In [ ]:
test = R.from_euler('zyx', data_all.values[0:1].values.reshape(23, 3), True)

In [ ]:
poses_unflat = {}

In [ ]:
for path, data in data_all:
    try:
        poses = data.values.values
        trace = poses[:, :3].reshape(poses.shape[0], 3)
        poses = poses[:, 3:].reshape(poses.shape[0], 22, 3)
        poses = np.array([R.from_euler('zyx', pose, True).as_matrix().reshape(22 * 3 * 3) for pose in poses])
        
        poses_unflat[path.split('/')[-1]] = np.concatenate([poses, trace], 1)
    except KeyboardInterrupt:
        raise
    except Exception as e:
        print(path, e)

In [ ]:
len(poses_unflat)

In [ ]:
real_samples_dataset_180 = []
real_samples_dataset_300 = []

In [ ]:
for key, value in poses_unflat.items():
    if 'subject5' in key:
        types = 'test'
    else:
        types = 'train'
    print(key)
    for split in np.array(range(value.shape[0]))[::300]:
        
        tmp = {}

        if value[split:split+180].shape[0] == 180:
            tmp['type'] = types
            tmp['pose'] = value[split:split+180]
            real_samples_dataset_180.append(tmp)

#             tmp = value[split:split+300]
        tmp = {}

        if value[split:split+300].shape[0] == 300:
            tmp['type'] = types
            tmp['pose'] = value[split:split+300]
            real_samples_dataset_300.append(tmp)

In [ ]:
real_samples_dataset_full = []

In [ ]:
for key, value in poses_unflat.items():
    if 'subject5' in key:
        types = 'test
    else:
        types = 'train'

    tmp = {}

    tmp['type'] = types
    tmp['pose'] = value
    real_samples_dataset_full.append(tmp)

In [ ]:
np.save('data/processed/ubisoft.rotation_matrix.full', real_samples_dataset_full)

In [ ]:
real_samples_dataset_300 = np.array(real_samples_dataset_300)

In [ ]:
real_samples_dataset_180 = np.array(real_samples_dataset_180)

In [ ]:
np.save('data/processed/ubisoft.rotation_matrix.300', real_samples_dataset_300)

In [ ]:
np.save('data/processed/ubisoft.rotation_matrix.180', real_samples_dataset_180)